# 05.5 XGBoost Scratch Demo

**Senior Data Scientist:** En este notebook, demostraremos la funcionalidad de nuestra implementación "Desde Cero" del algoritmo XGBoost. 
El objetivo es validar que nuestra lógica matemática (Gradientes, Hessianos, Ganancia Estructural) y el bucle de Boosting secuencial funcionan correctamente y pueden aprender patrones simples de los datos clínicos procesados.

Este hito corresponde al **Issue 14: Prototipo Desde Cero Funcional**.

## 1. Configuración del Entorno y Carga de Datos

Importamos las librerías necesarias y nuestra clase `XGBoostScratch`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import sys
import os

# Aseguramos que el path raíz esté accesible para importar src
if os.path.abspath("..") not in sys.path:
    sys.path.append(os.path.abspath(".."))

from src.tree.xgboost_scratch import XGBoostScratch

# Configuración de pandas
pd.set_option('display.max_columns', None)

## 2. Carga y Preparación de Datos

Cargamos el dataset procesado `process_data.parquet`. Para esta demostración, utilizaremos un subconjunto de características clave para mantener el entrenamiento rápido y fácil de interpretar.

In [2]:
# Cargar datos
data_path = "../data/02_intermediate/process_data.parquet"
df = pd.read_parquet(data_path)

print(f"Dimensiones originales: {df.shape}")

# Mapeo de nombres de columnas (Español -> Inglés) si es necesario
# El archivo de memoria indica que process_data tiene nombres en español que luego se renonmbran.
# Vamos a inspeccionar las columnas primero.
print(df.columns.tolist())

Dimensiones originales: (43695, 28)
['SEQN', 'Sex', 'Age', 'Race', 'Education', 'IncomeRatio', 'HeartDisease', 'SystolicBP', 'BMI', 'WaistCircumference', 'Height', 'TotalCholesterol', 'Triglycerides', 'LDL', 'HbA1c', 'Glucose', 'Creatinine', 'UricAcid', 'ALT_Enzyme', 'Albumin', 'Potassium', 'Sodium', 'GGT_Enzyme', 'AST_Enzyme', 'Smoking', 'PhysicalActivity', 'HealthInsurance', 'Alcohol']


In [3]:
# Seleccionamos features numéricas clave y el target 'HeartDisease'
# Nota: Ajustar nombres de columnas según la salida anterior si difieren.
# Asumimos que el target ya es 'HeartDisease' o 'TARGET'.

target_col = 'HeartDisease' if 'HeartDisease' in df.columns else 'TARGET'

# Features simplificadas para la demo
features = ['Age', 'SystolicBP', 'TotalCholesterol', 'Glucose', 'BMI']

# Verificamos si las columnas existen (o sus equivalentes en español)
# Si están en español, las renombramos para consistencia con el modelo scratch que no tiene el mapa interno.
rename_map = {
    'Edad': 'Age',
    'Presion_Sistolica': 'SystolicBP',
    'Colesterol_Total': 'TotalCholesterol',
    'Glucosa': 'Glucose',
    'IMC': 'BMI',
    'TARGET': 'HeartDisease'
}

df_renamed = df.rename(columns=rename_map)

# Filtramos solo las que existen
available_features = [f for f in features if f in df_renamed.columns]
print(f"Features seleccionadas: {available_features}")

X = df_renamed[available_features].values
y = df_renamed[target_col].values

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Features seleccionadas: ['Age', 'SystolicBP', 'TotalCholesterol', 'Glucose', 'BMI']
Train shape: (34956, 5), Test shape: (8739, 5)


## 3. Entrenamiento del Modelo Scratch

Instanciamos `XGBoostScratch` con parámetros conservadores para validar el funcionamiento. Usaremos `n_estimators=5` para ver el progreso paso a paso.

In [4]:
xgb_scratch = XGBoostScratch(
    n_estimators=10,
    learning_rate=0.1,
    max_depth=3,
    lambda_=1.0,
    gamma=0.0
)

print("Iniciando entrenamiento...")
xgb_scratch.fit(X_train, y_train)
print("Entrenamiento completado.")

Iniciando entrenamiento...


Entrenamiento completado.


## 4. Evaluación y Predicciones

Realizamos predicciones sobre el conjunto de prueba y evaluamos métricas básicas.

In [5]:
# Predicciones
y_pred_proba = xgb_scratch.predict_proba(X_test)
y_pred_class = xgb_scratch.predict(X_test, threshold=0.5)

# Métricas
acc = accuracy_score(y_test, y_pred_class)
auc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {acc:.4f}")
print(f"AUC-ROC: {auc:.4f}")

print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_class))

Accuracy: 0.9752
AUC-ROC: 0.8783

Matriz de Confusión:
[[8522    0]
 [ 217    0]]


## 5. Validación Individual

Probamos el modelo con un caso de ejemplo manual para verificar que la inferencia funciona correctamente.

In [6]:
sample_idx = 0
sample_X = X_test[sample_idx].reshape(1, -1)
sample_y = y_test[sample_idx]

prediction = xgb_scratch.predict_proba(sample_X)[0]
print(f"Datos del Paciente: {sample_X}")
print(f"Valor Real: {sample_y}")
print(f"Predicción Modelo (Probabilidad): {prediction:.4f}")
print(f"Predicción Modelo (Clase): {1 if prediction >= 0.5 else 0}")

Datos del Paciente: [[ 80.  144.  173.  139.   43.2]]
Valor Real: 0
Predicción Modelo (Probabilidad): 0.2441
Predicción Modelo (Clase): 0


## Conclusión

Hemos validado que la implementación `XGBoostScratch` entrena sin errores, reduce la función de pérdida (implícitamente a través del boosting) y genera predicciones coherentes (AUC > 0.5). Esto cumple con el requisito de un prototipo funcional desde cero.